# ⚡️ Extracting Transmission Line Reporting Data from FERC1

At the request of Shivani, I'm extracting FERC1 tranmission line expenditure data.

# 🔌 Connect to the database

In [1]:
# Import Packages
import os
import polars as pl
import ibis

# Connect to the PUDL SQLite Database
db_path = '/Users/jikaczmarski/acep/warehouse/federal-electric-sector/pudl.sqlite'
pudl = ibis.sqlite.connect(db_path)

# 🔋 Grab the transmission line data

In [2]:
ferc1_transmission_expenditures = pudl.table("core_ferc1__yearly_transmission_lines_sched422")
f1_trans = ferc1_transmission_expenditures.to_polars()

In [3]:
f1_trans.head()

record_id,utility_id_ferc1,report_year,start_point,end_point,operating_voltage_kv,designed_voltage_kv,supporting_structure_type,transmission_line_length_miles,transmission_line_and_structures_length_miles,num_transmission_circuits,conductor_size_and_material,capex_land,capex_other,capex_total,opex_operations,opex_maintenance,opex_rents,opex_total
str,i64,i64,str,str,f64,f64,str,f64,f64,i64,str,f64,f64,f64,f64,f64,f64,f64
"""f1_xmssn_line_1994_12_2_0_1""",294,1994,"""South Bessemer TS""","""Snowdoun TS""",500.0,0.0,"""Tower""",92.0,0.0,1,null,null,null,null,null,null,null,null
"""f1_xmssn_line_1994_12_2_0_2""",294,1994,"""Miller Steam Plant""","""Mississippi St.Line (TVA)""",500.0,0.0,"""Tower""",71.86,0.0,1,null,null,null,null,null,null,null,null
"""f1_xmssn_line_1994_12_2_0_3""",294,1994,"""Miller Steam Plant""","""Arkadelphia (TVA)""",500.0,0.0,"""Tower""",23.27,0.0,1,null,null,null,null,null,null,null,null
"""f1_xmssn_line_1994_12_2_0_4""",294,1994,"""Miller Steam Plant""","""South Bessemer""",500.0,0.0,"""Tower""",30.82,0.0,1,null,null,null,null,null,null,null,null
"""f1_xmssn_line_1994_12_2_0_5""",294,1994,"""Miller Steam Plant""","""Clay TS""",230.0,0.0,"""Tower""",31.58,0.0,1,null,null,null,null,null,null,null,null


# 🗻 Filter by Chugach Electric Association, Inc.

## 🔎 Finding Chugach in the data

In [4]:
ferc1_id = pudl.table("core_pudl__assn_ferc1_pudl_utilities")
ferc1_id = ferc1_id.to_polars()
ferc1_id.head()

utility_id_ferc1,utility_name_ferc1,utility_id_pudl
i64,str,i64
1,"""missing respondent 0""",6757
2,"""Blackstone Valley Electric Com…",44
3,"""Boston Edison Company""",45
4,"""Cambridge Electric Light Compa…",50
5,"""CANAL ELECTRIC COMPANY""",51


In [5]:
from polars_fuzzy_match import fuzzy_match_score

pattern = 'chugach'
fuzzy_search = (
    ferc1_id.with_columns(
        score=fuzzy_match_score(
            pl.col('utility_name_ferc1'),
            pattern,
        )
    )
    .filter(pl.col('score').is_not_null())
    .sort(by='score', descending=True)
)
fuzzy_search

/var/folders/5p/crts39y53l1gf5kjc_xvxcb0psl3b4/T/ipykernel_20326/982905930.py:1: DeprecationWarning: The `polars.type_aliases` module is deprecated. The type aliases have moved to the `polars._typing` module to explicitly mark them as private. Please define your own type aliases, or temporarily import from the `polars._typing` module. A public `polars.typing` module will be added in the future.
  from polars_fuzzy_match import fuzzy_match_score


utility_id_ferc1,utility_name_ferc1,utility_id_pudl,score
i64,str,i64,u32
393,"""Chugach Electric Association, …",63,192
264,"""Fitchburg Gas and Electric Lig…",119,107


# 🧮 Filter by Chugach Data

In [6]:
f1_trans_chugach = f1_trans.filter(pl.col('utility_id_ferc1') == pl.select(fuzzy_search)[0,0])
f1_trans_chugach

record_id,utility_id_ferc1,report_year,start_point,end_point,operating_voltage_kv,designed_voltage_kv,supporting_structure_type,transmission_line_length_miles,transmission_line_and_structures_length_miles,num_transmission_circuits,conductor_size_and_material,capex_land,capex_other,capex_total,opex_operations,opex_maintenance,opex_rents,opex_total
str,i64,i64,str,str,f64,f64,str,f64,f64,i64,str,f64,f64,f64,f64,f64,f64,f64
"""f1_xmssn_line_1994_12_202_0_1""",393,1994,"""Beluga""","""Point MacKenzie""",230.0,230.0,"""Tower""",42.5,0.0,1,null,null,null,null,null,null,null,null
"""f1_xmssn_line_1994_12_202_0_2""",393,1994,"""Beluga""","""Point MacKenzie""",230.0,230.0,"""Tower""",42.5,0.0,1,null,null,null,null,null,null,null,null
"""f1_xmssn_line_1994_12_202_0_3""",393,1994,"""Beluga""","""Point MacKenzie""",138.0,230.0,"""Tower""",42.5,0.0,1,null,null,null,null,null,null,null,null
"""f1_xmssn_line_1994_12_202_0_4""",393,1994,"""Point MacKenzie""","""Teeland""",230.0,230.0,"""Tower""",26.0,0.0,1,null,null,null,null,null,null,null,null
"""f1_xmssn_line_1994_12_202_0_5""",393,1994,"""Point MacKenzie""","""W. Terminal""",230.0,230.0,"""Tower""",7.3,0.0,1,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""transmission_line_statistics_4…",393,2021,null,null,138.0,138.0,"""Wood Pole""",null,0.72,null,null,null,null,null,null,null,null,null
"""transmission_line_statistics_4…",393,2021,"""Postmark""","""Postmark Jct""",138.0,138.0,"""Wood Pole""",0.72,null,2,null,null,null,null,null,null,null,null
"""transmission_line_statistics_4…",393,2021,"""36th Jct""","""International Jct""",138.0,138.0,"""Steel Pole""",1.08,null,1,null,null,null,null,null,null,null,null


# 📝 Write to CSV

In [7]:
f1_trans_chugach.write_csv(os.path.join(os.curdir, 'derived-data', 'chugach-transmission-costs.csv'))